# Notes
We have 4D data:
- The hut name
- The date of access
- The date of booking interest
- Number of available beds

How can we visualize this best?
- [ ] Aggregate by week

Research questions:
- Which huts are bottlenecks?

In [1]:
from pathlib import Path

import pandas as pd

In [2]:
DATA_DIR = Path("../daily/")

In [130]:
loc

PosixPath('../daily/2023-12-12.csv')

In [126]:
dfs = []
for loc in DATA_DIR.glob("*.csv"):
    df = pd.read_csv(loc)
    df = df.rename(columns={df.columns[0]: "name"})

    if loc.name.startswith("2023"): # only do 2024
        continue
        
    df["date"] = loc.stem
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df["week"] = pd.to_datetime(df["date"]).dt.isocalendar().week
df = df.drop(["date"], axis=1)
df = df.groupby(["name", "week"]).sum()

df.columns = pd.to_datetime(df.columns) # after group we only have dates as columns

In [131]:
df

2024-06-01  2024-06-02  2024-06-03  \
name                          week                                       
Auberge Gîte Bon Abri         1             49          49          49   
                              2             49          49          49   
                              3             66          66          66   
                              4             93          93          93   
                              5            217         217         217   
...                                        ...         ...         ...   
Rifugio Monte Bianco CAI Uget 16           294         159         151   
                              17           294         162         141   
                              18           294         161         140   
                              19           294         161         140   
                              20           126          69          60   

                                    2024-06-04  2024-06-05  2024-06-06  \
name                          week                                       
Auberge Gîte Bon Abri         1             49          49          49   
                              2             49          49          49   
                              3             66          66          66   
                              4             93          93          93   
                              5            217         217         217   
...                                        ...         ...         ...   
Rifugio Monte Bianco CAI Uget 16            51           0         147   
                              17            31           0         147   
                              18            24           0         159   
                              19            28           0         161   
                              20            12           0          69   

                                    2024-06-07  2024-06-08  2024-06-09  \
name                          week                                       
Auberge Gîte Bon Abri         1            105         224         140   
                              2            105         224         140   
                              3            109         211         139   
                              4            102         147         117   
                              5            238         343         273   
...                                        ...         ...         ...   
Rifugio Monte Bianco CAI Uget 16           147          36         182   
                              17           147          35         182   
                              18           147          13         176   
                              19           147          10         157   
                              20            63           3          66   

                                    2024-06-10  ...  2024-09-19  2024-09-20  \
name                          week              ...                           
Auberge Gîte Bon Abri         1            126  ...       387.0       287.0   
                              2            126  ...       385.0       287.0   
                              3            120  ...       330.0       246.0   
                              4            102  ...       165.0       120.0   
                              5            238  ...       385.0       280.0   
...                                        ...  ...         ...         ...   
Rifugio Monte Bianco CAI Uget 16           283  ...         0.0         0.0   
                              17           259  ...         0.0         0.0   
                              18           259  ...         0.0         0.0   
                              19           259  ...         0.0         0.0   
                              20           110  ...         0.0         0.0   

                                    2024-09-21  2024-09-22  2024-09-23  \
name                          week           

## What's the change per week?
- Negative value: beds are booked
- Positive value: beds are released

In [142]:
df_ = df.T
weekly_sum = df_.groupby(df_.index.isocalendar().week).sum()
df_by_week = weekly_sum.T
df_by_week.diff().to_csv("../tmp/test.csv")

In [129]:
df_by_week

week                                   22      23      24     25     26  \
name                          week                                        
Auberge Gîte Bon Abri         1      98.0   665.0  1281.0  986.0  788.0   
                              2      98.0   665.0  1226.0  880.0  622.0   
                              3     132.0   723.0  1146.0  709.0  477.0   
                              4     186.0   738.0   856.0  310.0  206.0   
                              5     434.0  1722.0  1967.0  658.0  384.0   
...                                   ...     ...     ...    ...    ...   
Rifugio Monte Bianco CAI Uget 16    453.0   714.0  1435.0  424.0  589.0   
                              17    456.0   683.0  1393.0  421.0  584.0   
                              18    455.0   659.0  1341.0  406.0  567.0   
                              19    455.0   643.0  1290.0  392.0  485.0   
                              20    195.0   273.0   534.0  158.0  180.0   

week                                   27     28     29      30      31  \
name                          week                                        
Auberge Gîte Bon Abri         1     688.0  901.0  875.0  1301.0  1186.0   
                              2     591.0  835.0  715.0  1309.0  1096.0   
                              3     490.0  641.0  568.0   938.0   889.0   
                              4     169.0  268.0  267.0   423.0   379.0   
                              5     287.0  563.0  561.0   892.0   796.0   
...                                   ...    ...    ...     ...     ...   
Rifugio Monte Bianco CAI Uget 16    857.0  658.0  519.0   359.0  1021.0   
                              17    788.0  658.0  496.0   387.0  1018.0   
                              18    763.0  640.0  476.0   396.0  1017.0   
                              19    736.0  629.0  464.0   358.0   970.0   
                              20    312.0  258.0  222.0   153.0   395.0   

week                                    32      33      34      35      36  \
name                          week                                           
Auberge Gîte Bon Abri         1     1534.0  1518.0  1805.0  1744.0  1531.0   
                              2     1448.0  1474.0  1750.0  1699.0  1451.0   
                              3     1189.0  1192.0  1339.0  1410.0  1218.0   
                              4      549.0   554.0   614.0   642.0   574.0   
                              5     1141.0  1154.0  1369.0  1404.0  1289.0   
...                                    ...     ...     ...     ...     ...   
Rifugio Monte Bianco CAI Uget 16     883.0   699.0  1084.0   571.0     0.0   
                              17     829.0   672.0  1053.0   582.0     0.0   
                              18     793.0   672.0   964.0   588.0     0.0   
                              19     761.0   670.0   881.0   588.0     0.0   
                              20     318.0   283.0   372.0   249.0     0.0   

week                                    37      38      39  
name                          week                          
Auberge Gîte Bon Abri         1     1193.0  1553.0     0.0  
                              2      936.0  1533.0     0.0  
                              3      734.0  1391.0   342.0  
                              4      289.0   934.0  1026.0  
                              5      628.0  2170.0  2378.0  
...                                    ...     ...     ...  
Rifugio Monte Bianco CAI Uget 16       0.0     0.0     0.0  
                              17       0.0     0.0     0.0  
                              18       0.0     0.0     0.0  
                              19       0.0     0.0     0.0  
                              20       0.0     0.0     0.0  

[800 rows x 18 columns]